In [ ]:
# Main program that performs the stock forecast 

In [2]:
# import libraries
import pandas as pd
import numpy as np
import datetime
import matplotlib. pyplot as plt
from pandas.tseries.offsets import BusinessDay
from keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Bidirectional
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.utils import timeseries_dataset_from_array
import plotly.graph_objects as go
import glob
import os

In [ ]:
### Functions ###

In [3]:
# get list of stock tickers in df
def get_ticker_list(df):
    tickers = [col.split('_')[1] for col in df.columns[1:]]
    count, i = 0, 0
    while i < len(tickers):
        if count == 0:
            unique = tickers[i]
            count+=1
            i+=1
        else:
            if tickers[i] == unique:
                tickers.pop(i)
            else:
                unique = tickers[i]
                count = 1
                i+=1
    return tickers

In [4]:
# computes relative strength index (rsi) from a stock's close prices
def get_rsi(prices):
    periods = 14
    ema = True
    close_delta = prices.diff()

    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    
    if ema == True:
        # Use exponential moving average
        ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    else:
        # Use simple moving average
        ma_up = up.rolling(window = periods, adjust=False).mean()
        ma_down = down.rolling(window = periods, adjust=False).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    return rsi

In [5]:
def get_return(df):
    returns = [np.NaN]
    for i in range(1,len(df)):
        returns.append(100*(df[i] - df[i-1])/df[i-1])
    return returns
        

In [6]:
# puts df data into a dictionary, adding rsi data, as well as train/test dates
def put_into_dict(df, tickers):
    dfdict = dict()
    split_percent = 0.90
    split = int(split_percent*len(df.index))
    # dictionary with full range open, close, vol, and rsi values
    for ticker in tickers:
        dfdict['open_%s' % (ticker)]  = df['open_%s' % (ticker)].values
        dfdict['close_%s' % (ticker)] = df['close_%s' % (ticker)].values
        dfdict['volume_%s' % (ticker)] = df['volume_%s' % (ticker)].values
        dfdict['rsi_%s' % (ticker)] = get_rsi(df['close_%s' % (ticker)])
    date_tr = df.index[:split]
    date_test = df.index[split:]
    return dfdict

In [7]:
def feature_scale(df, split):
    #scaler = StandardScaler()
    df_fs = pd.DataFrame()
    # avoid feature scaling date column
    #for col in df.columns[1:]:
        #df_fs[[col]] = scaler.fit_transform(df[[col]])
    df_fs.set_axis(df['date'], inplace = True)
    #return df_fs, scaler
    dict_scalers = dict()
    for col in df.columns[1:]:
        dict_scalers['%s_mean_tr'%(col)] = df[col].iloc[:split].mean() # store mean and std to access in inverse feature scaling later
        dict_scalers['%s_std_tr'%(col)] = df[col].iloc[:split].std()
        dict_scalers['%s_mean_test'%(col)] = df[col].iloc[split:].mean() 
        dict_scalers['%s_std_test'%(col)] = df[col].iloc[split:].std()
        df_fs[col] = 0
        for i, ind in enumerate(df.index):
            if i < split:
                df_fs[col].iloc[i] = (df[col].iloc[i] - df[col].iloc[:split].mean()) / df[col].iloc[:split].std()
            else:
                df_fs[col].iloc[i] = (df[col].iloc[i] - df[col].iloc[split:].mean())/ df[col].iloc[split:].std()
    return df_fs, dict_scalers


In [8]:
def inv_feature_scale(dfdict, tickers, dict_scalers):
    #for ticker in tickers:
        #dfdict['pred_%s'%(ticker.lower())] = scaler.inverse_transform(dfdict['pred_%s_fs'%(ticker.lower())])
    for ticker in tickers:
        dfdict['pred_%s'%(ticker)] = dfdict['pred_%s_fs'%(ticker)] * dict_scalers['close_%s_std_test'%(ticker)] + dict_scalers['close_%s_mean_test'%(ticker)]
    

In [9]:
def basic_LSTM_model(df, look_back, split=0.85, base = 'AAPL', num_epochs=30):
    base = base.lower()
    model = Sequential()
    
    model.add(
        LSTM(10, activation = 'relu', input_shape = (look_back, 1))
    )
    model.add(Dense(1))
    model.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])
    training = timeseries_dataset_from_array(df[['close_%s'%(base)]].to_numpy(), df[['close_%s'%(base)]].to_numpy(), sequence_length = look_back, 
                                                    batch_size = 12, start_index = None, end_index = int(split*len(df.index)))
    model.fit(training, epochs = num_epochs, verbose = 1)
    return model

In [42]:
def CNN_LSTM_model(df, split=0.90, base='AAPL',num_epochs=100, batch_size=16):
    base = base.lower()
    model=Sequential()
    model.add(Conv1D(filters=24, kernel_size=1, padding='same', activation='tanh',input_shape=(1, 6)))
    model.add(MaxPooling1D(pool_size=1, padding='same'))
    model.add(LSTM(24, activation='tanh'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    training = timeseries_dataset_from_array(df[['close_%s'%(base),'open_%s'%(base),'high_%s'%(base),
                                                'low_%s'%(base),'rsi_%s'%(base),'ret_%s'%(base)]],
                                            df[['close_%s'%(base)]], sequence_length=1, batch_size=batch_size,
                                            start_index=14, end_index=int(split*len(df.index)))
    model.fit(training, epochs=num_epochs,verbose=1)
    #plot_model(model, show_shapes=True, show_layer_names=True)
    return model

In [139]:
def construct_strategy(date, preds, act, split, run_as_test=True):
    n, i, profit = len(preds)-1, 0, 0
    while (i < (n - 1)):   
        # iterate until local minima is found in prediction set
        while ((i < (n - 1)) and (preds[i + 1] <= preds[i])): 
            i += 1
        # If we reached the end, break
        if (i == n - 1):
            break
        # Store the index of minima
        buy = i   
        # iterate to find local maxima
        while ((i < n) and (preds[i+1] >= preds[i])):
            i += 1
        if run_as_test:
            act_open = act
        if i<(n-1) and run_as_test:
            sell = i
          
            print("Buy after mkt open on", date[split+buy+1], " and sell after mkt open on", date[split+sell+1])
            print(" Predicted profit for trade: $",(preds[sell] - preds[buy]) * 10, "Actual profit for trade: $",(act_open[sell+1] - act_open[buy+1]) * 10)
            # Calculate the expected profit buying/selling 10 units
            profit += (act_open[sell+1] - act_open[buy+1]) * 10
    return '${:,.2f}'.format(profit)

In [12]:
# call with start_index = split-5 for standard prediction
def make_prediction(model, df, start_index, end_index=None, base = 'JNJ', look_back=1): 
    base = base.lower()
    trial = timeseries_dataset_from_array(df[['close_%s'%(base),'open_%s'%(base),'high_%s'%(base),
                                                'low_%s'%(base),'rsi_%s'%(base),'ret_%s'%(base)]], 
                                          None, sequence_length = look_back, 
                                          start_index = start_index, end_index = None)
    return model.predict(trial)

In [30]:
def plot_single_large(df, date, prediction, split, base='JNJ'):
    base = base.lower()
    trace1 = go.Scatter(
        x = df.index.values[400:split+1],
        y = df['close_%s'%(base)][400:split+1],
        mode = 'lines',
        name = 'Data'
    )
    trace2 = go.Scatter(
        x = date[split-1:],
        y = prediction.reshape(-1),
        mode = 'lines',
        name = 'Prediction'
    )
    trace3 = go.Scatter(
        x = df.index.values[split:],
        y = df['close_%s'%(base)][split:],
        mode = 'lines',
        name = 'Ground Truth'
    )
    layout = go.Layout(
        title = base.upper(),
        xaxis = {'title' : "Date"},
        yaxis = {'title' : "Close"}
    )
    fig = go.Figure(data = [trace1, trace2, trace3], layout = layout)
    fig.show()

In [ ]:
### Main Code Block ###

In [123]:
# Pulls updated (already pulled from db and api) stock data. Sets date to index
csv_name = 'XL_portfolio.csv'
df = pd.read_csv(csv_name)
#df = df.iloc[:,1:]
df.set_axis(df['Date'], inplace = True)
df.columns = [col.lower() for col in df.columns]
df.tail()

,date,open_xlb,close_xlb,high_xlb,low_xlb,volume_xlb,open_xlc,close_xlc,high_xlc,low_xlc,...,open_xlf,close_xlf,high_xlf,low_xlf,volume_xlf,open_xlk,close_xlk,high_xlk,low_xlk,volume_xlk
Date,,,,,,,,,,,,,,,,,,,,,
2022-07-28,2022-07-28,76.12,77.10,77.180,75.950,5915993.0,55.94,56.34,56.350,55.24,...,33.03,33.24,33.250,32.50,40974555.0,140.07,142.04,142.36,138.82,6204042.0
2022-07-29,2022-07-29,77.65,78.13,78.340,77.350,4629139.0,55.86,56.37,56.490,55.52,...,33.32,33.71,33.820,33.29,38724004.0,142.48,144.22,144.67,141.88,5088675.0
2022-08-01,2022-08-01,77.76,77.50,77.820,76.935,5205845.0,55.95,56.29,56.870,55.58,...,33.39,33.43,33.615,33.20,34943634.0,142.92,143.91,145.35,142.57,6258003.0
2022-08-02,2022-08-02,77.11,76.71,77.785,76.280,5683689.0,55.94,56.20,56.875,55.88,...,33.27,33.06,33.490,33.02,36760688.0,142.67,142.89,144.54,141.66,7960900.0
2022-08-03,2022-08-03,77.13,76.79,77.320,76.140,4798734.0,56.42,57.55,57.730,56.32,...,33.31,33.56,33.640,33.12,27280718.0,143.59,146.69,147.10,143.56,7121765.0


In [124]:
tickers = get_ticker_list(df)
print(tickers)

['xlb', 'xlc', 'xle', 'xli', 'xlf', 'xlk']


In [125]:
dfdict = put_into_dict(df, tickers)

In [126]:
for ticker in tickers:
    df['rsi_%s' % (ticker)] = get_rsi(df['close_%s' % (ticker)])
    df['ret_%s' % (ticker)] = get_return(df['close_%s' % (ticker)])

In [127]:
split = int(0.90 * len(df.index))

In [128]:
df_fs, dict_scaler = feature_scale(df, split) #ignore warning

C:\Users\HP\AppData\Local\Temp\ipykernel_18492\3867947128.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\HP\AppData\Local\Temp\ipykernel_18492\3867947128.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\HP\AppData\Local\Temp\ipykernel_18492\3867947128.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\HP\AppData\Local\Temp\ipykernel_18492\3867947128.py:18: SettingWithCopyWarning:


A value is tryi

C:\Users\HP\AppData\Local\Temp\ipykernel_18492\3867947128.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\HP\AppData\Local\Temp\ipykernel_18492\3867947128.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\HP\AppData\Local\Temp\ipykernel_18492\3867947128.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\HP\AppData\Local\Temp\ipykernel_18492\3867947128.py:18: SettingWithCopyWarning:


A value is tryi

In [129]:
df_fs.tail(10)

,open_xlb,close_xlb,high_xlb,low_xlb,volume_xlb,open_xlc,close_xlc,high_xlc,low_xlc,volume_xlc,...,rsi_xlc,ret_xlc,rsi_xle,ret_xle,rsi_xli,ret_xli,rsi_xlf,ret_xlf,rsi_xlk,ret_xlk
date,,,,,,,,,,,,,,,,,,,,,
2022-07-21,-0.662942,-0.448525,-0.559513,-0.575222,-0.921415,0.533818,0.685150,0.418101,0.465288,0.603814,...,1.925345,-0.029983,-0.237091,-0.628542,1.039612,0.409865,1.233241,0.372987,1.563178,0.583268
2022-07-22,-0.347300,-0.552738,-0.400947,-0.450029,-0.588869,0.138935,-0.287361,0.107202,-0.110435,-0.798976,...,0.497901,-1.693170,-0.338545,-0.265081,0.923244,-0.292489,1.007199,-0.390830,1.067089,-0.815900
2022-07-25,-0.500338,-0.492912,-0.589125,-0.442325,-1.316972,-0.196716,-0.379289,-0.366549,-0.215567,-1.264626,...,0.377389,-0.170123,0.179151,1.488977,1.057556,0.243838,1.213729,0.372742,0.848914,-0.441373
2022-07-26,-0.553902,-0.591336,-0.620647,-0.478920,-1.323144,-0.483006,-0.921186,-0.830431,-0.661126,-0.525291,...,-0.307061,-0.982673,0.052006,-0.298647,0.845095,-0.440622,0.660181,-0.904145,0.318400,-0.936873
2022-07-27,-0.678246,-0.417647,-0.475454,-0.594482,-0.376957,-0.418837,0.196475,0.072657,-0.180523,-0.245222,...,1.175252,2.053387,0.359626,0.937918,1.288139,0.968015,1.164206,0.935214,1.350613,2.036848
2022-07-28,-0.335822,-0.159044,-0.261486,-0.213126,-0.346495,-0.251012,-0.118018,-0.396159,-0.300674,-0.241613,...,0.771920,-0.560464,0.435668,0.275646,1.791558,1.334262,1.389577,0.426686,1.661531,0.641416
2022-07-29,-0.043136,0.039733,-0.039876,0.056519,-0.955377,-0.290500,-0.103503,-0.327070,-0.160498,0.384709,...,0.790519,0.021146,0.964425,1.750328,2.200666,1.312549,1.802914,0.857165,1.958075,0.647628
2022-08-01,-0.022093,-0.081849,-0.139218,-0.023411,-0.682505,-0.246076,-0.142210,-0.139544,-0.130460,-0.803091,...,0.736117,-0.073938,0.612701,-0.750742,2.220394,-0.006342,1.433111,-0.549940,1.876480,-0.230949
2022-08-02,-0.146437,-0.234309,-0.145905,-0.149567,-0.456411,-0.251012,-0.185755,-0.137076,0.019729,-0.305949,...,0.671199,-0.082691,0.574089,-0.035395,1.744974,-0.788574,0.970457,-0.723073,1.600892,-0.478911


In [ ]:
df_fs.info()

In [130]:
for i,ticker in enumerate(tickers):
    model = CNN_LSTM_model(df_fs, base=ticker, batch_size = 8)
    dfdict['pred_%s_fs'%(ticker)] = make_prediction(model, df_fs, split-1, base=ticker)
    print(f'finished model and prediction of stock {ticker.upper()} {i+1}/{len(tickers)}')
#model = CNN_LSTM_model(df_fs, base='QQQ', batch_size=1)


Epoch 1/100
55/55 [==============================] - 4s 5ms/step - loss: 0.7124
Epoch 2/100
55/55 [==============================] - 0s 6ms/step - loss: 0.1931
Epoch 3/100
55/55 [==============================] - 0s 6ms/step - loss: 0.0490
Epoch 4/100
55/55 [==============================] - 0s 7ms/step - loss: 0.0356
Epoch 5/100
55/55 [==============================] - 0s 6ms/step - loss: 0.0338
Epoch 6/100
55/55 [==============================] - 1s 10ms/step - loss: 0.0308
Epoch 7/100
55/55 [==============================] - 0s 6ms/step - loss: 0.0289
Epoch 8/100
55/55 [==============================] - 0s 6ms/step - loss: 0.0268
Epoch 9/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0251
Epoch 10/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0235
Epoch 11/100
55/55 [==============================] - 1s 9ms/step - loss: 0.0221
Epoch 12/100
55/55 [==============================] - 0s 7ms/step - loss: 0.0208
Epoch 13/100
55/55 [================

1/1 [==============================] - 1s 988ms/step
finished model and prediction of stock XLB 1/6
Epoch 1/100
55/55 [==============================] - 4s 3ms/step - loss: 0.7270
Epoch 2/100
55/55 [==============================] - 0s 3ms/step - loss: 0.1555
Epoch 3/100
55/55 [==============================] - 0s 4ms/step - loss: 0.0435
Epoch 4/100
55/55 [==============================] - 0s 9ms/step - loss: 0.0298
Epoch 5/100
55/55 [==============================] - 1s 10ms/step - loss: 0.0266
Epoch 6/100
55/55 [==============================] - 1s 10ms/step - loss: 0.0239
Epoch 7/100
55/55 [==============================] - 1s 10ms/step - loss: 0.0216
Epoch 8/100
55/55 [==============================] - 1s 10ms/step - loss: 0.0197
Epoch 9/100
55/55 [==============================] - 1s 9ms/step - loss: 0.0181
Epoch 10/100
55/55 [==============================] - 1s 9ms/step - loss: 0.0168
Epoch 11/100
55/55 [==============================] - 1s 10ms/step - loss: 0.0157
Epoch 12/100


1/1 [==============================] - 1s 937ms/step
finished model and prediction of stock XLC 2/6
Epoch 1/100
55/55 [==============================] - 4s 3ms/step - loss: 0.7284
Epoch 2/100
55/55 [==============================] - 0s 3ms/step - loss: 0.2876
Epoch 3/100
55/55 [==============================] - 0s 5ms/step - loss: 0.0933
Epoch 4/100
55/55 [==============================] - 0s 7ms/step - loss: 0.0629
Epoch 5/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0505
Epoch 6/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0424
Epoch 7/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0365
Epoch 8/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0321
Epoch 9/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0287
Epoch 10/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0260
Epoch 11/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0239
Epoch 12/100
55/55

1/1 [==============================] - 1s 820ms/step
finished model and prediction of stock XLE 3/6
Epoch 1/100
55/55 [==============================] - 4s 3ms/step - loss: 0.8512
Epoch 2/100
55/55 [==============================] - 0s 3ms/step - loss: 0.3543
Epoch 3/100
55/55 [==============================] - 0s 4ms/step - loss: 0.0986
Epoch 4/100
55/55 [==============================] - 0s 5ms/step - loss: 0.0378
Epoch 5/100
55/55 [==============================] - 0s 7ms/step - loss: 0.0300
Epoch 6/100
55/55 [==============================] - 0s 7ms/step - loss: 0.0264
Epoch 7/100
55/55 [==============================] - 0s 6ms/step - loss: 0.0232
Epoch 8/100
55/55 [==============================] - 0s 7ms/step - loss: 0.0209
Epoch 9/100
55/55 [==============================] - 1s 9ms/step - loss: 0.0190
Epoch 10/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0176
Epoch 11/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0164
Epoch 12/100
55/55

55/55 [==============================] - 0s 7ms/step - loss: 6.3182e-04
Epoch 100/100
1/1 [==============================] - 1s 820ms/step
finished model and prediction of stock XLI 4/6
Epoch 1/100
55/55 [==============================] - 4s 3ms/step - loss: 0.8440
Epoch 2/100
55/55 [==============================] - 0s 4ms/step - loss: 0.3203
Epoch 3/100
55/55 [==============================] - 0s 4ms/step - loss: 0.0902
Epoch 4/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0374
Epoch 5/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0294
Epoch 6/100
55/55 [==============================] - 1s 9ms/step - loss: 0.0252
Epoch 7/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0222
Epoch 8/100
55/55 [==============================] - 0s 7ms/step - loss: 0.0200
Epoch 9/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0181
Epoch 10/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0166
Epoch 11/100


55/55 [==============================] - 0s 6ms/step - loss: 0.0015
Epoch 100/100
1/1 [==============================] - 1s 1s/step
finished model and prediction of stock XLF 5/6
Epoch 1/100
55/55 [==============================] - 5s 4ms/step - loss: 0.7176
Epoch 2/100
55/55 [==============================] - 0s 8ms/step - loss: 0.2159
Epoch 3/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0665
Epoch 4/100
55/55 [==============================] - 0s 6ms/step - loss: 0.0494
Epoch 5/100
55/55 [==============================] - 0s 5ms/step - loss: 0.0432
Epoch 6/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0385
Epoch 7/100
55/55 [==============================] - 0s 5ms/step - loss: 0.0346
Epoch 8/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0312
Epoch 9/100
55/55 [==============================] - 0s 9ms/step - loss: 0.0284
Epoch 10/100
55/55 [==============================] - 0s 9ms/step - loss: 0.0261
Epoch 11/100
55/55 [

55/55 [==============================] - 0s 5ms/step - loss: 0.0032
Epoch 100/100
1/1 [==============================] - 1s 777ms/step
finished model and prediction of stock XLK 6/6


In [ ]:
#dfdict['pred_qqq_fs'] = make_prediction(model, df_fs, split-1, base='QQQ')

In [131]:
inv_feature_scale(dfdict, tickers, dict_scaler)
#inv_feature_scale(dfdict, ['aapl'], dict_scaler)


In [132]:
date_with_pred = pd.to_datetime(df['date'])
bd = BusinessDay(n=1)
date_with_pred = date_with_pred + bd
for ticker in tickers:
    plot_single_large(df, date_with_pred, dfdict['pred_%s'%(ticker)],split, base='%s'%(ticker.upper()))

In [140]:
profits = dict()
sum_for_10_shares = 0
for ticker in tickers: 
    total_profit = construct_strategy(date_with_pred, dfdict['pred_%s'%(ticker)], 
                                      df['open_%s'%(ticker)][split:], split, run_as_test=True)
    profits['Profit_%s'%(ticker)] = total_profit

for i, profit in enumerate(profits.values()):
    sum_for_10_shares += float(profit.split('$')[1])
    print(f'The profit for {tickers[i]} is {profit}')
print(f'The total profit in the period from {date_with_pred[split-1]} to {date_with_pred[-1]} is $ {sum_for_10_shares}')

Buy after mkt open on 2022-05-24 00:00:00  and sell after mkt open on 2022-05-26 00:00:00
 Predicted profit for trade: $ [40.795593] Actual profit for trade: $ -3.400000000000034
Buy after mkt open on 2022-05-27 00:00:00  and sell after mkt open on 2022-06-02 00:00:00
 Predicted profit for trade: $ [27.27829] Actual profit for trade: $ 17.69999999999996
Buy after mkt open on 2022-06-06 00:00:00  and sell after mkt open on 2022-06-07 00:00:00
 Predicted profit for trade: $ [12.984924] Actual profit for trade: $ 5.700000000000074
Buy after mkt open on 2022-06-08 00:00:00  and sell after mkt open on 2022-06-10 00:00:00
 Predicted profit for trade: $ [6.720047] Actual profit for trade: $ -11.200000000000045
Buy after mkt open on 2022-06-23 00:00:00  and sell after mkt open on 2022-06-24 00:00:00
 Predicted profit for trade: $ [8.294525] Actual profit for trade: $ 3.5999999999999943
Buy after mkt open on 2022-06-28 00:00:00  and sell after mkt open on 2022-06-29 00:00:00
 Predicted profit f